In [70]:
import pandas as pd
import numpy as np
from hazm import Normalizer, word_tokenize, Lemmatizer
import fasttext
from sklearn.cluster import KMeans
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the data from CSV
# If loading from a file:
df = pd.read_csv('questions_merged.csv')
df['ID'] = df.reset_index().index+1

df.head(5)

,Question,ID
0,من الگوی رگالی رو تهیه کردم و می‌خوام باهاش لب...,1
1,اگر همون خط جلو رو در نظر بگیرم، یقه خیلی از ب...,2
2,چرا من نمی تونم وارد سایت دوخت برتر بشم؟,3
3,برای من کد نمی یاد و پیامک نمی یاد، خانم حسینی...,4
4,چرا پنل‌ها قطع می‌شود؟ من الان وسط کارم بودم و...,5


In [71]:
# Step 1: Preprocess the questions using Hazm
normalizer = Normalizer(remove_specials_chars=False, decrease_repeated_chars=False)
lemmatizer = Lemmatizer(joined_verb_parts=False)

def preprocess_text(text):
    normalized = normalizer.normalize(text)
    tokens = word_tokenize(normalized)
    lemmatize = [lemmatizer.lemmatize(token) for token in tokens if len(token) > 1]
    return " ".join(lemmatize)

# Apply preprocessing
df['normelized'] = df['Question'].apply(preprocess_text)
df.head(4)

,Question,ID,normelized
0,من الگوی رگالی رو تهیه کردم و می‌خوام باهاش لب...,1,من الگو رگال رو تهیه کرد#کن می‌خوام با لباس جل...
1,اگر همون خط جلو رو در نظر بگیرم، یقه خیلی از ب...,2,اگر همون خط جلو رو در نظر گرفت#گیر یقه خیلی از...
2,چرا من نمی تونم وارد سایت دوخت برتر بشم؟,3,چرا من نمی‌تونم وارد سایت دوخت برتر بشم
3,برای من کد نمی یاد و پیامک نمی یاد، خانم حسینی...,4,برای من کد نمی‌یاد پیامک نمی‌یاد خانم حسینی گف...


In [72]:
import re

def remove_persian_prepositions(text):
    # لیست حروف اضافه و کلمات زائد
    persian_prepositions = [
        # حروف اضافه پایه
        'از', 'به', 'با', 'در', 'بر', 'برای', 'تا', 'بی', 'جز', 'را', 'که', 'یا', 'من', 'تو', 'او',
        'رو', 'این', 'هم', 'اگر', 'اگه',
        
        # حروف اضافه ترکیبی
        'بدون', 'مانند', 'همچون', 'بجز', 'درباره', 'پیرامون', 'میان',
        'روی', 'زیر', 'پشت', 'جلوی', 'کنار', 'نزد', 'پیش', 'پس از',
        'قبل از', 'بعد از', 'غیر از', 'علاوه بر', 'به جای', 'در مورد',
        'به سوی', 'به طرف', 'از طریق', 'از سوی', 'به وسیله', 'به واسطه',
        'به علت', 'به خاطر', 'به دلیل', 'با وجود', 'در مقابل', 'در برابر',
        'بر اساس', 'بر طبق', 'بر حسب', 'در صورت', 'در نتیجه',
        'سلام', '؟', '؟؟؟', 'اما', 'آن','چون','هر','اینکه','ولی'
    ]

    # ایجاد الگوی تطبیق برای حذف حروف اضافه
    pattern = r'\b(' + '|'.join(persian_prepositions) + r')\b'
    
    # جایگزینی الگو با رشته خالی
    cleaned_text = re.sub(pattern, '', text)

    # حذف فاصله‌های اضافی
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    return cleaned_text

# اعمال تابع روی ستون 'normelized' و ذخیره در 'Preposition'
df['Preposition'] = df['normelized'].apply(remove_persian_prepositions)
df.head(4)


,Question,ID,normelized,Preposition
0,من الگوی رگالی رو تهیه کردم و می‌خوام باهاش لب...,1,من الگو رگال رو تهیه کرد#کن می‌خوام با لباس جل...,الگو رگال تهیه کرد#کن می‌خوام لباس جلو بسته ما...
1,اگر همون خط جلو رو در نظر بگیرم، یقه خیلی از ب...,2,اگر همون خط جلو رو در نظر گرفت#گیر یقه خیلی از...,همون خط جلو نظر گرفت#گیر یقه خیلی بغل باز میشه...
2,چرا من نمی تونم وارد سایت دوخت برتر بشم؟,3,چرا من نمی‌تونم وارد سایت دوخت برتر بشم,چرا نمی‌تونم وارد سایت دوخت برتر بشم
3,برای من کد نمی یاد و پیامک نمی یاد، خانم حسینی...,4,برای من کد نمی‌یاد پیامک نمی‌یاد خانم حسینی گف...,کد نمی‌یاد پیامک نمی‌یاد خانم حسینی گفت#گو شما...


In [73]:
def remove_persian_verb(text):
    # List of common Persian prepositions
    persian_verb = [
        '#هست',
        'آمد#آ',
        'آموخت#آموز',
        'افتاد#افت',
        'افزود#افزا',
        'انجامید#انجام',
        'انداخت#انداز',
        'ایستاد#ایست',
        'ایستاند#ایستان',
        'بازآمد#بازآی',
        'بود#است',
        'بود#باش',
        'پذیرفت#پذیر',
        'توانست#توان',
        'جست#جوی',
        'چشید#چش',
        'داد#ده',
        'داشت#دار',
        'رسید#رس',
        'رفت#رو',
        'ساخت#ساز',
        'گذاشت#گذار',
        'گذارد#گذار',
        'گفت#گو',
        'گفت#گوی',
        'چرا',
        'چجوری',
        'چگونه',
        'چه',
        'کرد#کن',
        'چیست',
        'توانست#توان',
        'انجام',
        'دهم',
        'گرفت#گیر',
        'بشم',
        'دید#بین',
        'شد#شو',
        'زد#زن',
        'خواست#خواه',
        'میشه',        
        'داره',
        'دارد',
        'کشت#کش',
        'دانست#دان',        
        'خورد#خور',        
        'پرسید#پرس',        
        'دید#بببین',        
        'باشید#باش',        
        'خرید#خر',        
        'فرستاد#فرست',        
        'زارید#زار',        
        'نمود#نما',        
        'هست',
        'چرخید',
        'رست',
        'کشید#کش',
        'کاشت#کار',
        'لندید#لند',
        'برد',
        'هست',
        'بست#بند',
        'فهمید#فهم',
        'نداره',
        'ندارد',
        'شوم',
        'شم',
        'خواهید_کرد',
        'داشته_باشیم',
        'داشته_باشد',
        'چطور',
        'آیا',
        'باید',
        'است',
        'هم',
        'رو',
        'اگر',
        'ریخت#ریز',
        'گذشت#گذر',
        'نیازیدن#نیاز',
        'آورد#آور',
        'گوشید#گوش',
        'نخواهد_شد',
        'خواهد_بود',
        'نشده_است',
        'گشت#گرد',
        'باخت#باز',
        '؟',
        'خاست#خیز',
        'یافت#یاب',
        'فرمود#فرمای',
        'داده_می‌شود',
        'آمده_بود',
        'نوشت#نویس',
        'بخشید#بخش',
        'رساند#رسان',
        'یارست#یار',
        'نیست',
        'کدام',
        'کدوم',
        'دیده_می‌شود',
        'کشیده_می‌شود',
        'نیستند',
        'پوشیده_بود',
        'داشته_باشم',
        'گرفته_شود',
        'چیکار',
        'چکار',
        'لطفا',
        'باشه',
        'نمیشه',
        'کردن',
        'نمیخوام',
        'میخوام'
    ]

    # Create a pattern for prepositions with word boundaries
    # Use the \b (word boundary) to ensure we match whole words only
    pattern = r'\b(' + '|'.join(persian_verb) + r')\b'
    
    # Replace prepositions with empty string
    cleaned_text = re.sub(pattern, '', text)
    
    # Remove extra spaces that might be created
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()

    cleaned_text = cleaned_text.replace("#", "")
    cleaned_text = cleaned_text.replace("؟؟؟", "")
    return cleaned_text

df['preprocessed'] = df['Preposition'].apply(remove_persian_verb)
df.head(4)

,Question,ID,normelized,Preposition,preprocessed
0,من الگوی رگالی رو تهیه کردم و می‌خوام باهاش لب...,1,من الگو رگال رو تهیه کرد#کن می‌خوام با لباس جل...,الگو رگال تهیه کرد#کن می‌خوام لباس جلو بسته ما...,الگو رگال تهیه می‌خوام لباس جلو بسته مانتو جلو...
1,اگر همون خط جلو رو در نظر بگیرم، یقه خیلی از ب...,2,اگر همون خط جلو رو در نظر گرفت#گیر یقه خیلی از...,همون خط جلو نظر گرفت#گیر یقه خیلی بغل باز میشه...,همون خط جلو نظر یقه خیلی بغل باز نظر شما انداز...
2,چرا من نمی تونم وارد سایت دوخت برتر بشم؟,3,چرا من نمی‌تونم وارد سایت دوخت برتر بشم,چرا نمی‌تونم وارد سایت دوخت برتر بشم,نمی‌تونم وارد سایت دوخت برتر
3,برای من کد نمی یاد و پیامک نمی یاد، خانم حسینی...,4,برای من کد نمی‌یاد پیامک نمی‌یاد خانم حسینی گف...,کد نمی‌یاد پیامک نمی‌یاد خانم حسینی گفت#گو شما...,کد نمی‌یاد پیامک نمی‌یاد خانم حسینی شما جوابگو...


In [74]:
df.to_csv('preproces.csv', index=False)

**کلمات پرتکرار20**

In [75]:
# Step 2: Create keyword frequency analysis
def extract_keywords(texts):
    all_words = []
    for text in texts:
        words = text.split()
        all_words.extend(words)
    
    # Count frequency and filter stopwords
    word_counts = Counter(all_words)
    common_words = word_counts.most_common(100)
    return common_words

keywords = extract_keywords(df['preprocessed'])
print(keywords)

[('الگو', 42785), ('سایز', 12970), ('یقه', 12942), ('دوره', 10317), ('دوختدوز', 9017), ('قسمت', 8961), ('دوخت', 8863), ('دور', 8746), ('آستین', 8015), ('استفاده', 7613), ('کت', 7405), ('سانت', 7298), ('شلوار', 7284), ('رگال', 6768), ('جلو', 6751), ('پارچه', 6503), ('برش', 6336), ('لباس', 6160), ('بالاتنه', 6108), ('کمر', 6074), ('آموزش', 5608), ('اندازه', 5571), ('یک', 5494), ('سینه', 5315), ('خط', 4956), ('باسن', 4888), ('مشکل', 4658), ('کار', 4240), ('پنس', 4120), ('مدل', 4102), ('سرشانه', 4045), ('حلقه', 4039), ('مانتو', 3975), ('اضافه', 3872), ('آماده', 3804), ('باز', 3712), ('استاد', 3504), ('دامن', 3498), ('لقی', 3182), ('پایین', 3176), ('قد', 2777), ('رسم', 2741), ('راهنمایی', 2736), ('خیلی', 2719), ('بدم', 2712), ('گرد', 2709), ('وقتی', 2654), ('کم', 2651), ('فقط', 2621), ('فری', 2620), ('دو', 2590), ('بشه', 2509), ('تبدیل', 2481), ('پهلو', 2477), ('تهیه', 2426), ('شده', 2426), ('میتون', 2413), ('چقدر', 2396), ('ساسون', 2396), ('ایراد', 2329), ('چند', 2246), ('مورد', 2197), ('آ

In [ ]:
# Step 2: Create keyword frequency analysis
def extract_keywords(texts):
    all_words = []
    for text in texts:
        words = text.split()
        all_words.extend(words)
    
    # Count frequency and filter stopwords
    word_counts = Counter(all_words)
    common_words = word_counts.most_common(20)
    return common_words

keywords = extract_keywords(df['preprocessed'])

# Step 3: Use TF-IDF for feature extraction (as an alternative to FastText)
vectorizer = TfidfVectorizer(min_df=1, max_df=0.9)
tfidf_matrix = vectorizer.fit_transform(df['preprocessed'])

# Step 4: Calculate similarity between questions
similarity_matrix = cosine_similarity(tfidf_matrix)

# Step 5: Cluster the questions
num_clusters = 5  # Adjust based on your data size
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df['cluster'] = kmeans.fit_predict(tfidf_matrix.toarray())
tfidf_matrix

In [106]:
# Step 6: Find the most representative questions from each cluster
def find_central_questions(df, tfidf_matrix, kmeans, n_per_cluster=10):
    central_questions = []
    
    for cluster_id in range(num_clusters):
        cluster_indices = df[df['cluster'] == cluster_id].index
        
        if len(cluster_indices) == 0:
            continue
            
        # Calculate mean vector for this cluster
        cluster_center = kmeans.cluster_centers_[cluster_id]
        
        # Find distances to center
        distances = []
        for idx in cluster_indices:
            question_vector = tfidf_matrix[idx].toarray().flatten()
            distance = np.linalg.norm(question_vector - cluster_center)
            distances.append((idx, distance))
        
        # Sort by distance and get closest ones
        distances.sort(key=lambda x: x[1])
        closest_indices = [idx for idx, _ in distances[:min(n_per_cluster, len(distances))]]
        
        # Add to our list of central questions
        for idx in closest_indices:
            central_questions.append((df.iloc[idx]['ID'], df.iloc[idx]['Question'], cluster_id))
    
    return central_questions

# Get representative questions
central_questions = find_central_questions(df, tfidf_matrix, kmeans)

# Print results
print("Top representative questions by cluster:")
for id, question, cluster in central_questions:
    print(f"Cluster {cluster} - ID {id}: {question}")

# Find most common keywords
print("\nMost common keywords:")
for word, count in keywords:
    print(f"{word}: {count}")


Top representative questions by cluster:
Cluster 0 - ID 266: کل اندازه هام رو بفرستم یا دور کمر و باسن و دور سینه؟
Cluster 0 - ID 32: من می خواستم از روی جدول سایز الگو بکشم دور کمر دور باسن من سایز 42 دور سینه سایز 40 بقیه اندازه ها هم از سایز 36 هست تا 44 گیج شدم.
Cluster 0 - ID 142: با دور سینه ۱۰۰، دور کمر ۹۲، و دور باسن ۹۶، تو جدول سایز باید کدومو انتخاب کنم؟
Cluster 0 - ID 20: برای مشتری که دور سینه 107، کمر 97 و باسن 117 هستش، با چه سایزی الگو بکشم و چه ایرادهایی بگیرم؟
Cluster 0 - ID 17: چرا سرشانه الگوی تجاری که می‌کشم خیلی بزرگ میشه و با سایز 38 و دور سینه 107، کمر 94 و باسن 116 چطور و با چه سایزی الگو بکشم؟
Cluster 0 - ID 219: با توجه به اندازه‌های دور باسن ۱۲۷ و دور کمر ۱۱۰، چه تغییراتی برای برش الگو باید انجام دهم؟
Cluster 0 - ID 218: در مورد الگوهای آماده، دور سینه من ۱۲۰ هست، چه نکاتی باید در نظر بگیرم؟
Cluster 0 - ID 252: دور شکم خیلی خوب شد ولی دور سینه اضافی داشت. اگر بخواهم به شکم و باسن آزادی بدهم، می‌توانم از باسن شروع کنم و به کمر ختم کنم؟ اینجوری الگو خراب نمی‌شو

In [107]:
# Create a visualization of question clusters (optional)
def visualize_clusters(tfidf_matrix, clusters):
    from sklearn.decomposition import PCA
    
    # Reduce to 2D for visualization
    pca = PCA(n_components=2)
    coords = pca.fit_transform(tfidf_matrix.toarray())
    
    # Plot
    plt.figure(figsize=(10, 8))
    
    for cluster_id in range(num_clusters):
        cluster_points = coords[clusters == cluster_id]
        plt.scatter(cluster_points[:, 0], cluster_points[:, 1], label=f'Cluster {cluster_id}')
    
    plt.legend()
    plt.title('Question Clusters Visualization')
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.savefig('question_clusters.png')
    plt.close()

# Uncomment to visualize (requires matplotlib)
# visualize_clusters(tfidf_matrix, df['cluster'].values)

# For a full dataset, you would process all 1000 questions and select the top 50
# Return the ids of the top questions sorted by representativeness


In [108]:
def get_top_50_questions(central_questions, total_needed=50):
    # Sort questions by cluster to ensure variety
    clusters = {}
    for id, question, cluster in central_questions:
        if cluster not in clusters:
            clusters[cluster] = []
        clusters[cluster].append((id, question))
    
    # Take questions from each cluster in a round-robin fashion
    top_questions = []
    while len(top_questions) < total_needed and any(clusters.values()):
        for cluster in sorted(clusters.keys()):
            if clusters[cluster]:
                top_questions.append(clusters[cluster].pop(0))
                if len(top_questions) >= total_needed:
                    break
    
    return top_questions

# With a full dataset, you would use this to get your final 50 questions
if len(central_questions) > 50:
    top_50 = get_top_50_questions(central_questions)
    print("\nTop 50 Representative Questions:")
    for i, (id, question) in enumerate(top_50):
        print(f"{i+1}. ID {id}: {question}")
else:
    print("\nAll Representative Questions (sample data):")
    for i, (id, question, _) in enumerate(central_questions):
        print(f"{i+1}. ID {id}: {question}")


All Representative Questions (sample data):
1. ID 266: کل اندازه هام رو بفرستم یا دور کمر و باسن و دور سینه؟
2. ID 32: من می خواستم از روی جدول سایز الگو بکشم دور کمر دور باسن من سایز 42 دور سینه سایز 40 بقیه اندازه ها هم از سایز 36 هست تا 44 گیج شدم.
3. ID 142: با دور سینه ۱۰۰، دور کمر ۹۲، و دور باسن ۹۶، تو جدول سایز باید کدومو انتخاب کنم؟
4. ID 20: برای مشتری که دور سینه 107، کمر 97 و باسن 117 هستش، با چه سایزی الگو بکشم و چه ایرادهایی بگیرم؟
5. ID 17: چرا سرشانه الگوی تجاری که می‌کشم خیلی بزرگ میشه و با سایز 38 و دور سینه 107، کمر 94 و باسن 116 چطور و با چه سایزی الگو بکشم؟
6. ID 219: با توجه به اندازه‌های دور باسن ۱۲۷ و دور کمر ۱۱۰، چه تغییراتی برای برش الگو باید انجام دهم؟
7. ID 218: در مورد الگوهای آماده، دور سینه من ۱۲۰ هست، چه نکاتی باید در نظر بگیرم؟
8. ID 252: دور شکم خیلی خوب شد ولی دور سینه اضافی داشت. اگر بخواهم به شکم و باسن آزادی بدهم، می‌توانم از باسن شروع کنم و به کمر ختم کنم؟ اینجوری الگو خراب نمی‌شود؟
9. ID 296: برای الگوی دامن تنگ با دور باسن ۱۰۸ و دور کمر ۹۲، برای